<a href="https://colab.research.google.com/github/MohamedElgharbawy/Z3Performance/blob/main/Z3_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plotting Z3 Performance

This program plots Z3's performance when solving theorems with n random variables and a random set of [1, n<sup>2</sup>] constraints.

In [2]:
from z3 import *
import random
import operator
import time
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import os
import errno
import stopit

In [38]:
# Utils
ops = [operator.lt, operator.le, operator.ge, operator.gt]
twenty_min = 1200 # seconds

def write_to_dir(filename, contents):
  """
  Creates directory if it doesn't exist, then writes contents to it
  """
  if not os.path.exists(os.path.dirname(filename)):
    try:
        os.makedirs(os.path.dirname(filename))
    except OSError as exc:
        if exc.errno != errno.EEXIST:
          raise

  with open(filename, "w") as f:
    f.write(contents)

def try_check_model(solver, timeout):
  """ 
  input: solver, timeout (seconds)
  output: satisfiability (sat/unsat)
  throws: TimeoutError
  """
  with stopit.SignalTimeout(timeout) as to_ctx_mgr:
    return solver.check()

  if to_ctx_mgr.state != to_ctx_mgr.EXECUTED:
    raise TimeoutError(f"Solver timed out after {timeout} seconds.")
    

In [31]:
def z3_performance(num_var: int, use_max: bool, timeout: int, inc_constraints=1):
  """
  Measure Z3's performance to check a model for num_var variables with
  1 - num_var**2 constraints.
  Returns the a tuple, a list of the number of constraints, and a list of the
  corresponding times.
  """
  X = IntVector('x', num_var)

  if use_max:
    solver = Optimize()
    solver.maximize(Sum(X))
  else:
    solver = Solver()

  # Graphing lists
  num_constraints = []
  times = []
  num_unsat = 0

  # Constraints
  constraints = []
  for i in range(1, num_var**2 + 1, inc_constraints):
    print(f"{num_var}: {i} constraints")
    num_constraints.append(i)

    # Get a random operator
    op = random.choice(ops)
    
    # Create a random constraint
    constraint = op(sum(random.randint(-10, 10)*X[j] for j in range(0,num_var)), random.randint(-10, 10))
    constraints.append(constraint)
    solver.add(constraint)

    # Write problem to smt lib txt file
    filename = f"problems/{num_var}/{num_var},num_constraints={i}.txt"
    write_to_dir(filename, solver.sexpr())

    # Check model
    start_time = time.monotonic()
    satisfiability = None
    try:
      satisfiability = try_check_model(solver, timeout)
      final_time = time.monotonic() - start_time
      times.append(round(final_time, 5))
    except:
      times.append(np.nan)

    filename = f"models/{num_var}/{num_var},num_constraints={i}.txt"
    if satisfiability == "unsat":
      num_unsat += 1
      write_to_dir(filename, "unsat")
    elif satisfiability == "sat":
      write_to_dir(filename, solver.model().sexpr())
    else:
      write_to_dir(filename, "timeout")
    
    if num_unsat == 10:
      break

  return num_constraints, times

In [46]:
def plot_one_z3_performance(num_var: int, use_max: bool, timeout: int, inc_constraints=1):
  """
  Measure and plot Z3's performance for num_var variables
  """
  num_constraints, times = z3_performance(num_var, use_max, timeout, inc_constraints)
  
  if np.nan in times:
    plt.axhline(y=timeout, color="red", linestyle='-', label="timeout")

  # for x, y in zip(num_constraints, times):
  #   if np.isnan(y):
  #     plt.text(x, timeout, "x", fontsize=12, color="red", ha="center", va="center")

  # Replace nan times with timeout before plotting
  times = np.nan_to_num(times, nan=timeout)
  plt.plot(num_constraints, times)

  plt.title(f"Z3 Performance for {num_var} variables")
  plt.xlabel("Number of constraints")
  plt.ylabel("Time (seconds)")
  plt.legend()
  plt.savefig(f"plots/{num_var},use_max={use_max}")
  plt.clf()

In [45]:
random.seed(42)
plot_one_z3_performance(25, use_max=False, timeout=1, inc_constraints=5)

25: 1 constraints
25: 6 constraints
25: 11 constraints
25: 16 constraints
25: 21 constraints
25: 26 constraints
25: 31 constraints
25: 36 constraints
25: 41 constraints
25: 46 constraints
25: 51 constraints
25: 56 constraints
25: 61 constraints
25: 66 constraints
25: 71 constraints
25: 76 constraints
25: 81 constraints
25: 86 constraints
25: 91 constraints
25: 96 constraints
25: 101 constraints
25: 106 constraints
25: 111 constraints
25: 116 constraints
25: 121 constraints
25: 126 constraints
25: 131 constraints
25: 136 constraints
25: 141 constraints
25: 146 constraints
25: 151 constraints
25: 156 constraints
25: 161 constraints
25: 166 constraints
25: 171 constraints
25: 176 constraints
25: 181 constraints
25: 186 constraints
25: 191 constraints
25: 196 constraints
25: 201 constraints
25: 206 constraints
25: 211 constraints
25: 216 constraints
25: 221 constraints
25: 226 constraints
25: 231 constraints
25: 236 constraints
25: 241 constraints
25: 246 constraints
25: 251 constraints
25

In [40]:
random.seed(42)
# plot_one_z3_performance(50, use_max=True, inc_constraints=10)
plot_one_z3_performance(50, use_max=False, timeout=10, inc_constraints=10)

TypeError: plot_one_z3_performance() missing 1 required positional argument: 'timeout'

In [43]:
random.seed(42)
plot_one_z3_performance(5, use_max=False, timeout=twenty_min)

5: 1 constraints
5: 2 constraints
5: 3 constraints
5: 4 constraints
5: 5 constraints
5: 6 constraints
5: 7 constraints
5: 8 constraints
5: 9 constraints
5: 10 constraints
5: 11 constraints
5: 12 constraints
5: 13 constraints
5: 14 constraints
5: 15 constraints
5: 16 constraints
5: 17 constraints
5: 18 constraints
5: 19 constraints
5: 20 constraints
5: 21 constraints
5: 22 constraints
5: 23 constraints
5: 24 constraints
5: 25 constraints
